In [1]:
print("Helllo Jupyter")

Helllo Jupyter


In [5]:
!pip install tensorflow

     ------------------------------------ 276.5/276.5 MB 967.1 kB/s eta 0:00:00
     -------------------------------------- 126.5/126.5 KB 2.5 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 1.7 MB/s eta 0:00:00
     -------------------------------------- 440.8/440.8 KB 2.1 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 KB 1.5 MB/s eta 0:00:00
     ---------------------------------------- 4.3/4.3 MB 1.7 MB/s eta 0:00:00
     ---------------------------------------- 2.7/2.7 MB 1.4 MB/s eta 0:00:00
     -------------------------------------- 430.5/430.5 KB 1.7 MB/s eta 0:00:00
     ---------------------------------------- 14.9/14.9 MB 1.8 MB/s eta 0:00:00
     ---------------------------------------- 24.4/24.4 MB 1.5 MB/s eta 0:00:00
     ---------------------------------------- 5.6/5.6 MB 1.9 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 1.8 MB/s eta 0:00:00
     ---------------------------------------- 65.5

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
filelock 3.12.3 requires typing-extensions>=4.7.1; python_version < "3.11", but you have typing-extensions 4.5.0 which is incompatible.
You should consider upgrading via the 'C:\Users\HP\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [3]:
import tensorflow as tf
import os
from os.path import isfile, join
import numpy as np
import shutil
from tensorflow import keras
from pathlib import Path
from IPython.display import display, Audio
import subprocess

In [4]:
data_directory = "./datasets"
audio_folder = "audio"
noise_folder = "_background_noise_"

audio_path = os.path.join(data_directory, audio_folder)
noise_path = os.path.join(data_directory, noise_folder)

In [5]:
audio_path

'./datasets\\audio'

In [6]:
valid_split = 0.1

shuffle_seed = 43

sample_rate = 16000

scale = 0.5

batch_size = 40 #changed from 127

epochs = 8 #changed from 15

In [7]:
for folder in os.listdir(data_directory):
    if os.path.isdir(os.path.join(data_directory, folder)):
        if folder in [audio_folder, noise_folder]:
            
            continue
        elif folder in ["other", "_background_noise_"]:
            
            shutil.move(
                os.path.join(data_directory, folder),
                os.path.join(noise_path, folder),
            )
        else:
            shutil.move(
                os.path.join(data_directory, folder),
                os.path.join(audio_path, folder),
            )

In [8]:
noise_paths = []
for subdir in os.listdir(noise_path):
    subdir_path = Path(noise_path) / subdir
    if os.path.isdir(subdir_path):
        noise_paths += [
            os.path.join(subdir_path, filepath)
            for filepath in os.listdir(subdir_path)
            if filepath.endswith(".wav")
        ]

In [9]:
noise_paths

['datasets\\_background_noise_\\other\\exercise_bike.wav',
 'datasets\\_background_noise_\\other\\pink_noise.wav']

In [10]:
command = (
    "for dir in `ls -1 " + noise_path + "`; do "
    "for file in `ls -1 " + noise_path + "/$dir/*.wav`; do "
    "sample_rate=`ffprobe -hide_banner -loglevel panic -show_streams "
    "$file | grep sample_rate | cut -f2 -d=`; "
    "if [ $sample_rate -ne 16000 ]; then "
    "ffmpeg -hide_banner -loglevel panic -y "
    "-i $file -ar 16000 temp.wav; "
    "mv temp.wav $file; "
    "fi; done; done"
)

In [11]:
os.system(command)
def load_noise_sample(path):
    sample, sampling_rate = tf.audio.decode_wav(
        tf.io.read_file(path), desired_channels=1
    )
    if sampling_rate == sample_rate:
        slices = int(sample.shape[0] / sample_rate)
        sample = tf.split(sample[: slices * sample_rate], slices)
        return sample
    else:
        print("Sampling rate for",path, "is incorrect")
        return None


noises = []
for path in noise_paths:
    sample = load_noise_sample(path)
    if sample:
        noises.extend(sample)
noises = tf.stack(noises)

Sampling rate for datasets\_background_noise_\other\exercise_bike.wav is incorrect
Sampling rate for datasets\_background_noise_\other\pink_noise.wav is incorrect


In [12]:
def paths_and_labels_to_dataset(audio_paths, labels):
    path_ds = tf.data.Dataset.from_tensor_slices(audio_paths)
    audio_ds = path_ds.map(lambda x: path_to_audio(x))
    label_ds = tf.data.Dataset.from_tensor_slices(labels)
    return tf.data.Dataset.zip((audio_ds, label_ds))

In [13]:
def path_to_audio(path):
    audio = tf.io.read_file(path)
    audio, _ = tf.audio.decode_wav(audio, 1, sample_rate)
    return audio

In [14]:
def add_noise(audio, noises=None, scale=0.5):
    if noises is not None:
        tf_rnd = tf.random.uniform(
            (tf.shape(audio)[0],), 0, noises.shape[0], dtype=tf.int32
        )
        noise = tf.gather(noises, tf_rnd, axis=0)

        prop = tf.math.reduce_max(audio, axis=1) / tf.math.reduce_max(noise, axis=1)
        prop = tf.repeat(tf.expand_dims(prop, axis=1), tf.shape(audio)[1], axis=1)

        audio = audio + noise * prop * scale

    return audio

In [15]:
def audio_to_fft(audio):
    audio = tf.squeeze(audio, axis=-1)
    fft = tf.signal.fft(
        tf.cast(tf.complex(real=audio, imag=tf.zeros_like(audio)), tf.complex64)
    )
    fft = tf.expand_dims(fft, axis=-1)

    return tf.math.abs(fft[:, : (audio.shape[1] // 2), :])

In [16]:
class_names = os.listdir(audio_path)
print(class_names,)

audio_paths = []
labels = []
for label, name in enumerate(class_names):
    print("Speaker:",(name))
    dir_path = Path(audio_path) / name
    speaker_sample_paths = [
        os.path.join(dir_path, filepath)
        for filepath in os.listdir(dir_path)
        if filepath.endswith(".wav")
    ]
    audio_paths += speaker_sample_paths
    labels += [label] * len(speaker_sample_paths)

['aleena', 'areeba', 'Benjamin_Netanyau', 'Jens_Stoltenberg', 'Julia_Gillard', 'Magaret_Tarcher', 'nabeeha', 'Nelson_Mandela']
Speaker: aleena
Speaker: areeba
Speaker: Benjamin_Netanyau
Speaker: Jens_Stoltenberg
Speaker: Julia_Gillard
Speaker: Magaret_Tarcher
Speaker: nabeeha
Speaker: Nelson_Mandela


In [17]:
# Shuffle to generate random data
rng = np.random.RandomState(shuffle_seed)
rng.shuffle(audio_paths)
rng = np.random.RandomState(shuffle_seed)
rng.shuffle(labels)

In [18]:
# Split into training and validation
num_val_samples = int(valid_split * len(audio_paths))
train_audio_paths = audio_paths[:-num_val_samples]
train_labels = labels[:-num_val_samples]


valid_audio_paths = audio_paths[-num_val_samples:]
valid_labels = labels[-num_val_samples:]

In [19]:
# Create datasets, one for training and the other for validation
train_ds = paths_and_labels_to_dataset(train_audio_paths, train_labels)
train_ds = train_ds.shuffle(buffer_size=batch_size * 8, seed=shuffle_seed).batch(
    batch_size
)

valid_ds = paths_and_labels_to_dataset(valid_audio_paths, valid_labels)
valid_ds = valid_ds.shuffle(buffer_size=32 * 8, seed=shuffle_seed).batch(32)

In [20]:
# Add noise to the training set
train_ds = train_ds.map(
    lambda x, y: (add_noise(x, noises, scale=scale), y),
    num_parallel_calls=tf.data.experimental.AUTOTUNE,
)

# Transform audio wave to the frequency domain using `audio_to_fft`
train_ds = train_ds.map(
    lambda x, y: (audio_to_fft(x), y), num_parallel_calls=tf.data.experimental.AUTOTUNE
)

train_ds = train_ds.prefetch(tf.data.experimental.AUTOTUNE)

valid_ds = valid_ds.map(
    lambda x, y: (audio_to_fft(x), y), num_parallel_calls=tf.data.experimental.AUTOTUNE
)
valid_ds = valid_ds.prefetch(tf.data.experimental.AUTOTUNE)


ValueError: in user code:

    File "C:\Users\HP\AppData\Local\Temp\ipykernel_4624\4097213244.py", line 3, in None  *
        lambda x, y: (add_noise(x, noises, scale=scale), y)
    File "C:\Users\HP\AppData\Local\Temp\ipykernel_4624\1548865019.py", line 8, in add_noise  *
        prop = tf.math.reduce_max(audio, axis=1) / tf.math.reduce_max(noise, axis=1)

    ValueError: Invalid reduction dimension 1 for input with 1 dimensions. for '{{node Max_1}} = Max[T=DT_FLOAT, Tidx=DT_INT32, keep_dims=false](GatherV2, Max_1/reduction_indices)' with input shapes: [?], [] and with computed input tensors: input[1] = <1>.


In [ ]:
from tensorflow.keras.layers import Conv1D

In [21]:
def residual_block(x, filters, conv_num = 3, activation = "relu"):
    s = keras.layers.Conv1D(filters, 1, padding = "same")(x)
    
    for i in range(conv_num - 1):
        x = keras.layers.Conv1D(filters, 3, padding = "same")(x)
        x = keras.layers.Activation(activation)(x)
    
    x = keras.layers.Conv1D(filters, 3, padding = "same")(x)
    x = keras.layers.Add()([x, s])
    x = keras.layers.Activation(activation)(x)
    
    return keras.layers.MaxPool1D(pool_size = 2, strides = 2)(x)

def build_model(input_shape, num_classes):
    inputs = keras.layers.Input(shape = input_shape, name = "input")
    
    # Adjust the Conv1D layer to accept the new input shape
    x = keras.layers.Conv1D(16, 3, padding="same")(inputs)
    
    x = residual_block(inputs, 16, 2)
    x = residual_block(inputs, 32, 2)
    x = residual_block(inputs, 64, 3)
    x = residual_block(inputs, 128, 3)
    x = residual_block(inputs, 128, 3)
    x = keras.layers.AveragePooling1D(pool_size=3, strides=3)(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(256, activation="relu")(x)
    x = keras.layers.Dense(128, activation="relu")(x)
    
    outputs = keras.layers.Dense(num_classes, activation = "softmax", name = "output")(x)
    
    return keras.models.Model(inputs = inputs, outputs = outputs)


# Define the new input shape (16000 samples)
new_input_shape = (16000, 1)

# Build the model with the new input shape
model = build_model(new_input_shape, len(class_names))

model.summary()

model.compile(optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]) 

model_save_filename = "model.h5"

earlystopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

mdlcheckpoint_cb = keras.callbacks.ModelCheckpoint(model_save_filename, monitor="val_accuracy", save_best_only=True)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 16000, 1)]           0         []                            
                                                                                                  
 conv1d_16 (Conv1D)          (None, 16000, 128)           512       ['input[0][0]']               
                                                                                                  
 activation_10 (Activation)  (None, 16000, 128)           0         ['conv1d_16[0][0]']           
                                                                                                  
 conv1d_17 (Conv1D)          (None, 16000, 128)           49280     ['activation_10[0][0]']       
                                                                                              

In [22]:
history = model.fit(
    train_ds,
    epochs=epochs,
    validation_data=valid_ds,
    callbacks=[earlystopping_cb, mdlcheckpoint_cb],
)

Epoch 1/8
169/169 [==============================] - ETA: 0s - loss: 0.9899 - accuracy: 0.5830

C:\Users\HP\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


169/169 [==============================] - 900s 5s/step - loss: 0.9899 - accuracy: 0.5830 - val_loss: 0.6222 - val_accuracy: 0.7027
Epoch 2/8
169/169 [==============================] - 875s 5s/step - loss: 0.4164 - accuracy: 0.8271 - val_loss: 0.4138 - val_accuracy: 0.8307
Epoch 3/8
169/169 [==============================] - 941s 6s/step - loss: 0.2708 - accuracy: 0.8871 - val_loss: 0.3854 - val_accuracy: 0.8493
Epoch 4/8
169/169 [==============================] - 926s 5s/step - loss: 0.1793 - accuracy: 0.9314 - val_loss: 0.4613 - val_accuracy: 0.8400
Epoch 5/8
  8/169 [>.............................] - ETA: 14:31 - loss: 0.1136 - accuracy: 0.9594

KeyboardInterrupt: 

In [ ]:
print("Accuracy of model:",model.evaluate(valid_ds))